In [1]:
!pip install tensorflow opencv-python


Defaulting to user installation because normal site-packages is not writeable


'''CNN MODEL'''

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os

# Define a simple CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes: fire, smoke, sparkle
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset
def prepare_dataset(data_dir, target_size=(64, 64), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (64, 64))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke', 'Sparkle']
    return classes[np.argmax(prediction)]

# Example usage
input_shape = (64, 64, 3)
cnn_model = create_cnn_model(input_shape)
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset'
train_gen, val_gen = prepare_dataset(data_dir)
train_model(cnn_model, train_gen, val_gen)

# Save the trained model
cnn_model.save('abnormality_classifier.h5')

C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 142 images belonging to 3 classes.
Found 33 images belonging to 3 classes.
Epoch 1/10


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.3226 - loss: 0.9203 - val_accuracy: 0.4545 - val_loss: 0.7180
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.7508 - loss: 0.7898 - val_accuracy: 0.4545 - val_loss: 0.7220
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.8449 - loss: 0.5916 - val_accuracy: 0.4545 - val_loss: 0.7580
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.8456 - loss: 0.5728 - val_accuracy: 0.4545 - val_loss: 1.4072
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.8476 - loss: 0.4601 - val_accuracy: 0.4545 - val_loss: 1.4851
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.8275 - loss: 0.4559 - val_accuracy: 0.4545 - val_loss: 1.7736
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.8532 - loss: 0.3794 - val_accuracy: 0.4545 - val_loss: 1.9625
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.8826 - loss: 0.2903 - val_accuracy: 0.4545 - val_loss: 2.2186
Epoch 9/10

In [3]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
from tensorflow.keras.models import load_model

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(model, frame_resized)
        output_frame_path = os.path.join(output_dir, f"abnormality_{abnormality_group_id}_{abnormality_class}", f"frame_{frame_number}.jpg")
        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)
    
    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the trained model
    model = load_model('abnormality_classifier.h5')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/cnntrail1'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

In [5]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
from tensorflow.keras.models import load_model

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(model, frame_resized)
        output_frame_path = os.path.join(output_dir, f"abnormality_{abnormality_group_id}_{abnormality_class}", f"frame_{frame_number}.jpg")
        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)
    
    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the trained model
    model = load_model('abnormality_classifier.h5')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/cnntrail2'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Abnormality 1 start time: 1 minutes 17.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 1 minutes 60.00 seconds
Abnormality detected at frame numbe

'''EfficientNetB0 as the base model for transfer learning, which is a state-of-the-art convolutional neural network (CNN) architecture that achieves a good balance between performance and computational efficiency. EfficientNet is a popular choice for image classification tasks, including detecting objects like fire and smoke.'''

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os

# Define a simple CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes: fire, smoke, sparkle
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset
def prepare_dataset(data_dir, target_size=(64, 64), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (64, 64))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke', 'Sparkle']
    return classes[np.argmax(prediction)]

# Example usage
input_shape = (64, 64, 3)
cnn_model = create_cnn_model(input_shape)
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset'
train_gen, val_gen = prepare_dataset(data_dir)
train_model(cnn_model, train_gen, val_gen)

# Save the trained model
cnn_model.save('abnormality_classifier2.h5')

Found 1767 images belonging to 3 classes.
Found 440 images belonging to 3 classes.
Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 156ms/step - accuracy: 0.9461 - loss: 0.3155 - val_accuracy: 0.9477 - val_loss: 0.1334
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 142ms/step - accuracy: 0.9761 - loss: 0.0774 - val_accuracy: 0.9568 - val_loss: 0.1184
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 140ms/step - accuracy: 0.9815 - loss: 0.0671 - val_accuracy: 0.9477 - val_loss: 0.1537
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 145ms/step - accuracy: 0.9750 - loss: 0.0906 - val_accuracy: 0.9477 - val_loss: 0.1228
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 141ms/step - accuracy: 0.9775 - loss: 0.0630 - val_accuracy: 0.9591 - val_loss: 0.1373
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 143ms/step - accuracy: 0.9816 - loss: 0.0573 - val_accuracy: 0.9682 - val_loss: 0.1022
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 139ms/step - accuracy: 0.9820 - loss: 0.0536 - val_accuracy: 0.9591 - val_loss: 0.1295
Epoch 8/10
56/56 ━━━

In [7]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
from tensorflow.keras.models import load_model

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(model, frame_resized)
        output_frame_path = os.path.join(output_dir, f"abnormality_{abnormality_group_id}_{abnormality_class}", f"frame_{frame_number}.jpg")
        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)
    
    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the trained model
    model = load_model('abnormality_classifier2.h5')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/cnntrail3'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Abnormality 1 start time: 1 minutes 17.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 1 minutes 60.00 seconds
Abnormality detected at frame number

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import cv2
import os

# Define the model using EfficientNetB0
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Number of classes
    ])
    base_model.trainable = False  # Freeze the base model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset
def prepare_dataset(data_dir, target_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['fire', 'smoke', 'sparkle']
    return classes[np.argmax(prediction)]

# Example usage
input_shape = (224, 224, 3)
num_classes = 3
cnn_model = create_efficientnet_model(input_shape, num_classes)
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset' # Update this to the path of your dataset
train_gen, val_gen = prepare_dataset(data_dir)
train_model(cnn_model, train_gen, val_gen, epochs=10)

# Save the trained model
cnn_model.save('abnormality_classifier_efficientnet3.h5')

# Example frame classification
frame = cv2.imread("C:/Users/pkathi/Desktop/object detection/Data/test/images/frame_103434.jpg")  # Replace with the path to a test frame
classification = classify_abnormality(cnn_model, frame)
print(f"Classified as: {classification}")


Found 1767 images belonging to 3 classes.
Found 440 images belonging to 3 classes.
Epoch 1/10


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


56/56 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.8760 - loss: 0.4585 - val_accuracy: 0.9477 - val_loss: 0.2094
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.9446 - loss: 0.2252 - val_accuracy: 0.9477 - val_loss: 0.2071
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 53s 929ms/step - accuracy: 0.9441 - loss: 0.2224 - val_accuracy: 0.9477 - val_loss: 0.2067
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 55s 975ms/step - accuracy: 0.9487 - loss: 0.2136 - val_accuracy: 0.9477 - val_loss: 0.2066
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.9334 - loss: 0.2531 - val_accuracy: 0.9477 - val_loss: 0.2087
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 56s 993ms/step - accuracy: 0.9519 - loss: 0.2017 - val_accuracy: 0.9477 - val_loss: 0.2086
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.9403 - loss: 0.2314 - val_accuracy: 0.9477 - val_loss: 0.2062
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.9379 - loss: 0.2424 - val_accuracy: 0.9477 - val_loss: 0

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Classified as: smoke


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import cv2
import os
from multiprocessing.pool import ThreadPool

# Define the model using EfficientNetB0
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Number of classes
    ])
    base_model.trainable = False  # Freeze the base model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset
def prepare_dataset(data_dir, target_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['fire', 'smoke', 'fire+smoke', 'sparkle']
    return classes[np.argmax(prediction)]

# Process frame with classification
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(model, frame)
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        os.makedirs(abnormality_dir, exist_ok=True)  # Ensure directory exists
        output_frame_path = os.path.join(abnormality_dir, f"{abnormality_class}_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        abnormality_time = frame_number / original_fps
        timestamp_path = os.path.join(abnormality_dir, f"{abnormality_class}_timestamps.txt")
        with open(timestamp_path, 'a') as f:
            f.write(f"{abnormality_class} - {abnormality_time:.2f} seconds\n")
    
    return detected, frame_gray

# Motion detection function
def motion_detection(video_path, output_dir, model_path, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    model = load_model(model_path)

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/effiecient4'
model_path = 'abnormality_classifier_efficientnet3.h5'  # Path to your trained model
abnormality_times = motion_detection(video_path, output_dir, model_path, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import cv2
import os
from multiprocessing.pool import ThreadPool

# Define the model using EfficientNetB0
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Number of classes
    ])
    base_model.trainable = False  # Freeze the base model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset
def prepare_dataset(data_dir, target_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['fire', 'smoke', 'fire+smoke', 'sparkle']
    return classes[np.argmax(prediction)]

# Process frame with classification
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(model, frame)
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        os.makedirs(abnormality_dir, exist_ok=True)  # Ensure directory exists
        output_frame_path = os.path.join(abnormality_dir, f"{abnormality_class}_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        abnormality_time = frame_number / original_fps
        timestamp_path = os.path.join(abnormality_dir, f"{abnormality_class}_timestamps.txt")
        with open(timestamp_path, 'a') as f:
            f.write(f"{abnormality_class} - {abnormality_time:.2f} seconds\n")
    
    return detected, frame_gray

# Motion detection function
def motion_detection(video_path, output_dir, model_path, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    model = load_model(model_path)

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/effiecient5'
model_path = 'abnormality_classifier_efficientnet3.h5'  # Path to your trained model
abnormality_times = motion_detection(video_path, output_dir, model_path, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Abnormality 1 start time: 1 minutes 17.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 1 minutes 60.00 seconds
Abnormality detected at frame numbers:

In [ ]:
#above giving only smoke--make efficient

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam

# Define the model using EfficientNetB0
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Number of classes
    ])
    base_model.trainable = False  # Freeze the base model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset
def prepare_dataset(data_dir, target_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)
    model.save('abnormality_classifier_efficientnet2.h5')

# Example usage
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset' # Replace with your dataset directory
input_shape = (224, 224, 3)
num_classes = 3

model = create_efficientnet_model(input_shape, num_classes)
train_generator, validation_generator = prepare_dataset(data_dir)
train_model(model, train_generator, validation_generator)


Found 142 images belonging to 3 classes.
Found 33 images belonging to 3 classes.
Epoch 1/10


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.4885 - loss: 1.0271 - val_accuracy: 0.4545 - val_loss: 0.8632
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 969ms/step - accuracy: 0.4758 - loss: 0.8511 - val_accuracy: 0.5455 - val_loss: 0.7621
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5873 - loss: 0.7855 - val_accuracy: 0.5455 - val_loss: 0.7270
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5471 - loss: 0.7536 - val_accuracy: 0.5455 - val_loss: 0.7104
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.4094 - loss: 0.7797 - val_accuracy: 0.4545 - val_loss: 0.7098
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 882ms/step - accuracy: 0.5028 - loss: 0.7583 - val_accuracy: 0.5455 - val_loss: 0.7014
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 866ms/step - accuracy: 0.5318 - loss: 0.7605 - val_accuracy: 0.5455 - val_loss: 0.7030
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 870ms/step - accuracy: 0.5116 - loss: 0.7773 - val_accuracy: 0.5455 - val_loss: 0.7121
Epoch 9/10
5/5

In [17]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Load the trained model
model = load_model('abnormality_classifier_efficientnet2.h5')
classes = ['fire', 'smoke', 'sparkle']

def classify_abnormality(frame, model):
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    return classes[np.argmax(prediction)]

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(frame, model)
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        os.makedirs(abnormality_dir, exist_ok=True)  # Ensure directory exists
        output_frame_path = os.path.join(abnormality_dir, f"{abnormality_class}_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        abnormality_time = frame_number / original_fps
        timestamp_path = os.path.join(abnormality_dir, f"{abnormality_class}_timestamps.txt")
        with open(timestamp_path, 'a') as f:
            f.write(f"{abnormality_class} - {abnormality_time:.2f} seconds\n")
    
    return detected, frame_gray

def motion_detection(video_path, output_dir, model_path, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    model = load_model(model_path)

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/effiecient7'
model_path = 'abnormality_classifier_efficientnet2.h5'  # Path to your trained model
abnormality_times = motion_detection(video_path, output_dir, model_path, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers", abnormality_times)
else:
    print("No abnormality detected.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Abnormality 1 start time: 1 minutes 17.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 1 minutes 60.00 seconds
Abnormality detected at frame numbers

In [18]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Load the trained model
model = load_model('abnormality_classifier_efficientnet2.h5')
classes = ['fire', 'smoke', 'sparkle']

def classify_abnormality(frame, model):
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    return classes[np.argmax(prediction)]

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(frame, model)
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        os.makedirs(abnormality_dir, exist_ok=True)  # Ensure directory exists
        output_frame_path = os.path.join(abnormality_dir, f"{abnormality_class}_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        abnormality_time = frame_number / original_fps
        timestamp_path = os.path.join(abnormality_dir, f"{abnormality_class}_timestamps.txt")
        with open(timestamp_path, 'a') as f:
            f.write(f"{abnormality_class} - {abnormality_time:.2f} seconds\n")
    
    return detected, frame_gray

def motion_detection(video_path, output_dir, model_path, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    model = load_model(model_path)

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\noabnormality.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/effiecient7'
model_path = 'abnormality_classifier_efficientnet2.h5'  # Path to your trained model
abnormality_times = motion_detection(video_path, output_dir, model_path, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers", abnormality_times)
else:
    print("No abnormality detected.")

Total video time: 0 minutes 60.00 seconds
No abnormality detected.


In [20]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Load the trained model
model = load_model('abnormality_classifier_efficientnet2.h5')
classes = ['fire', 'smoke', 'sparkle']

def classify_abnormality(frame, model):
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    return classes[np.argmax(prediction)]

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(frame, model)
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        os.makedirs(abnormality_dir, exist_ok=True)  # Ensure directory exists
        output_frame_path = os.path.join(abnormality_dir, f"{abnormality_class}_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        abnormality_time = frame_number / original_fps
        timestamp_path = os.path.join(abnormality_dir, f"{abnormality_class}_timestamps.txt")
        with open(timestamp_path, 'a') as f:
            f.write(f"{abnormality_class} - {abnormality_time:.2f} seconds\n")
    
    return detected, frame_gray

def motion_detection(video_path, output_dir, model_path, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    model = load_model(model_path)

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\2-DMG_RSA_HTE101_D_HS_FIRE_31-33min\output31.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/effiecient8'
model_path = 'abnormality_classifier_efficientnet2.h5'  # Path to your trained model
abnormality_times = motion_detection(video_path, output_dir, model_path, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers", abnormality_times)
else:
    print("No abnormality detected.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Abnormality 1 start time: 0 minutes 51.00 seconds
Abnormality 1 duration: 0 minutes 1.00 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Abnormality 1 start time: 0 minutes 52.80 seconds
Abnormality 1 duration: 0 minutes 0.60 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Abnormality 1 start time: 0 minutes 55.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam

# Define the model using EfficientNetB0
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Number of classes
    ])
    base_model.trainable = False  # Freeze the base model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset
def prepare_dataset(data_dir, target_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=25):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)
    model.save('abnormality_classifier_efficientnet3.h5')

# Example usage
data_dir = r"C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset" # Replace with your dataset directory
input_shape = (224, 224, 3)
num_classes = 2

model = create_efficientnet_model(input_shape, num_classes)
train_generator, validation_generator = prepare_dataset(data_dir)
train_model(model, train_generator, validation_generator)


Found 219 images belonging to 2 classes.
Found 54 images belonging to 2 classes.
Epoch 1/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.4777 - loss: 0.6896 - val_accuracy: 0.6111 - val_loss: 0.6694
Epoch 2/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5763 - loss: 0.6835 - val_accuracy: 0.6111 - val_loss: 0.6702
Epoch 3/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6233 - loss: 0.6692 - val_accuracy: 0.6111 - val_loss: 0.6689
Epoch 4/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6048 - loss: 0.6766 - val_accuracy: 0.6111 - val_loss: 0.6692
Epoch 5/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5943 - loss: 0.6862 - val_accuracy: 0.6111 - val_loss: 0.6693
Epoch 6/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6110 - loss: 0.6703 - val_accuracy: 0.6111 - val_loss: 0.6688
Epoch 7/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6218 - loss: 0.6632 - val_accuracy: 0.6111 - val_loss: 0.6687
Epoch 8/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accura

In [29]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Load the trained model
model = load_model('abnormality_classifier_efficientnet2.h5')
classes = ['fire', 'smoke', 'sparkle']

def classify_abnormality(frame, model):
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    return classes[np.argmax(prediction)]

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_class = classify_abnormality(frame, model)
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        os.makedirs(abnormality_dir, exist_ok=True)  # Ensure directory exists
        output_frame_path = os.path.join(abnormality_dir, f"{abnormality_class}_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        abnormality_time = frame_number / original_fps
        timestamp_path = os.path.join(abnormality_dir, f"{abnormality_class}_timestamps.txt")
        with open(timestamp_path, 'a') as f:
            f.write(f"{abnormality_class} - {abnormality_time:.2f} seconds\n")
    
    return detected, frame_gray

def motion_detection(video_path, output_dir, model_path, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    model = load_model(model_path)

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\2-DMG_RSA_HTE101_D_HS_FIRE_31-33min\output31.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/effiecient9'
model_path = 'abnormality_classifier_efficientnet3.h5'  # Path to your trained model
abnormality_times = motion_detection(video_path, output_dir, model_path, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers", abnormality_times)
else:
    print("No abnormality detected.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Abnormality 1 start time: 0 minutes 51.00 seconds
Abnormality 1 duration: 0 minutes 1.00 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Abnormality 1 start time: 0 minutes 52.80 seconds
Abnormality 1 duration: 0 minutes 0.60 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Abnormality 1 start time: 0 minutes 55.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━

'''With DATA AUGMENTATION'''

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os

# Define a simple CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # 2 classes: fire, smoke
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the dataset with data augmentation
def prepare_dataset(data_dir, target_size=(64, 64), batch_size=32):
    # Data augmentation parameters
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = validation_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (64, 64))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Example usage
input_shape = (64, 64, 3)
cnn_model = create_cnn_model(input_shape)
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset2'
train_gen, val_gen = prepare_dataset(data_dir)
train_model(cnn_model, train_gen, val_gen)

# Save the trained model
cnn_model.save('abnormality_classifier_augment2.h5')

# Verify the dataset preparation
print(f"Classes found: {train_gen.class_indices}")
print(f"Number of training samples: {train_gen.samples}")
print(f"Number of validation samples: {val_gen.samples}")


Found 228 images belonging to 2 classes.
Found 56 images belonging to 2 classes.


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 195ms/step - accuracy: 0.5775 - loss: 0.6844 - val_accuracy: 0.7143 - val_loss: 0.6068
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.6873 - loss: 0.6095 - val_accuracy: 0.7143 - val_loss: 0.5956
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.6705 - loss: 0.6267 - val_accuracy: 0.7143 - val_loss: 0.6340
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - accuracy: 0.7430 - loss: 0.5411 - val_accuracy: 0.7143 - val_loss: 0.6134
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - accuracy: 0.7312 - loss: 0.5465 - val_accuracy: 0.7143 - val_loss: 0.5936
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - accuracy: 0.7221 - loss: 0.5223 - val_accuracy: 0.7143 - val_loss: 0.5931
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.7026 - loss: 0.4842 - val_accuracy: 0.7143 - val_loss: 0.8677
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.6884 - loss: 0.6162 - val_accuracy: 0.7143 - val_loss:

Classes found: {'Fire': 0, 'Smoke': 1}
Number of training samples: 228
Number of validation samples: 56


In [12]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool

# Load the trained model
cnn_model = load_model('abnormality_classifier_augment2.h5')

# Define function to classify abnormality
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (64, 64))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        output_frame_path = os.path.join(output_dir, f"abnormality_{abnormality_group_id}", f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        classification = classify_abnormality(cnn_model, frame_resized)
        print(f"Abnormality classified as: {classification} at frame {frame_number}")

    return detected, frame_gray

# Motion detection function remains the same
def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/augmrnt2_0'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
Abnormality classified as: Fire at frame 1350
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Abnormality classified as: Fire at frame 1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Abnormality classified as: Fire at frame 1362
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Abnormality classified as: Fire at frame 1368
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Abnormality classified as: Fire at frame 1374
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Abnormality classified as: Fire at frame 1380
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Abnormality classified as: Fire at frame 1386
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Abnormality classified as: Fire at frame 1392
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Abnormality classified as: Fire at frame 1398
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Abnormality classified as: Fire at frame 1404
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Abnormality classified as: Fire at frame 1410
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Abnormality classified as: Fire at fr

In [14]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool

# Load the trained model
cnn_model = load_model('abnormality_classifier_augment2.h5')

# Define function to classify abnormality
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (64, 64))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(cnn_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function
def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/augment2_1'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Fire detected at 0m_45.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Fire detected at 0m_45.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Fire detected at 0m_45.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Fire detected at 0m_45.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Fire detected at 0m_45.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Fire detected at 0m_46.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Fire detected at 0m_46.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Fire detected at 0m_46.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Fire detected at 0m_46.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Fire detected at 0m_46.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Fire detected at 0m_47.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Fire detected at 0m_47.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Fire detected at 0m_47.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Fire detected at 0m_47.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Fire detected at 0m_47.80s
1/1 ━━━━━━━━━━━━━━━━━━━━

#Data Augmentation and Transfer Learning

'''VGG16'''

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from sklearn.utils.class_weight import compute_class_weight

# Load and preprocess dataset with data augmentation
def prepare_dataset(data_dir, target_size=(128, 128), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = validation_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Define the model using transfer learning (VGG16)
def create_transfer_learning_model(input_shape, num_classes):
    base_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model with class weighting
def train_model(model, train_generator, validation_generator, epochs=20):
    # Extract class indices from the training generator
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    classes = np.array([class_indices[i] for i in train_generator.classes])
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
    class_weights_dict = dict(enumerate(class_weights))

    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        class_weight=class_weights_dict
    )

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (128, 128))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Example usage
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset2'
input_shape = (128, 128, 3)
num_classes = 2

train_gen, val_gen = prepare_dataset(data_dir, target_size=(128, 128))
cnn_model = create_transfer_learning_model(input_shape, num_classes)
train_model(cnn_model, train_gen, val_gen)

# Save the trained model
cnn_model.save('abnormality_classifier_vgg16.h5')

# Verify the dataset preparation
print(f"Classes found: {train_gen.class_indices}")
print(f"Number of training samples: {train_gen.samples}")
print(f"Number of validation samples: {val_gen.samples}")

# Now use the saved model for classifying abnormalities in video frames
cnn_model = load_model('abnormality_classifier_vgg16.h5')

# Update the process_frame function to include classification and saving frames
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(cnn_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function remains the same
def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_end_time = frame_number / original_fps
                abnormality_duration_seconds = abnormality_end_time - (abnormality_start_frame / original_fps)

                print(f"Abnormality {abnormality_group_id} start time: {int((abnormality_start_frame / original_fps) // 60)} minutes {(abnormality_start_frame / original_fps) % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

                abnormality_detected = False

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/augmentvgg_0'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Found 228 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
Epoch 1/20


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step - accuracy: 0.5524 - loss: 0.9504 - val_accuracy: 0.4821 - val_loss: 1.0380
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.5102 - loss: 0.7178 - val_accuracy: 0.6607 - val_loss: 0.5400
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.7161 - loss: 0.7091 - val_accuracy: 0.4821 - val_loss: 0.7313
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step - accuracy: 0.6420 - loss: 0.5366 - val_accuracy: 0.5000 - val_loss: 0.6765
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.7755 - loss: 0.4529 - val_accuracy: 0.5893 - val_loss: 0.5434
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.7784 - loss: 0.4527 - val_accuracy: 0.5536 - val_loss: 0.6962
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.7656 - loss: 0.4279 - val_accuracy: 0.5893 - val_loss: 0.6002
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.7911 - loss: 0.4069 - val_accuracy: 0.5536 - val_loss: 0.8482
Epoch 9/20
8/8 ━━━━

Classes found: {'Fire': 0, 'Smoke': 1}
Number of training samples: 228
Number of validation samples: 56


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step
Fire detected at 0m_45.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Fire detected at 0m_45.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
Fire detected at 0m_45.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
Fire detected at 0m_45.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
Smoke detected at 0m_45.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
Fire detected at 0m_46.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
Fire detected at 0m_46.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
Fire detected at 0m_46.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Fire detected at 0m_46.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
Fire detected at 0m_46.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
Fire detected at 0m_47.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Fire detected at 0m_47.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
Fire detected at 0m_47.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
Fire detected at 0m_47.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
Fire detected at 0m_47.80s
1/1 ━━━━━

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Load and preprocess dataset with data augmentation
def prepare_dataset(data_dir, target_size=(128, 128), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = validation_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Define the model using transfer learning (VGG16)
def create_transfer_learning_model(input_shape, num_classes):
    base_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model with class weighting
def train_model(model, train_generator, validation_generator, epochs=20):
    # Extract class indices from the training generator
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    classes = np.array([class_indices[i] for i in train_generator.classes])
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
    class_weights_dict = dict(enumerate(class_weights))

    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        class_weight=class_weights_dict
    )

# Example usage
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset2'
input_shape = (128, 128, 3)
num_classes = 2

train_gen, val_gen = prepare_dataset(data_dir, target_size=(128, 128))
cnn_model = create_transfer_learning_model(input_shape, num_classes)
train_model(cnn_model, train_gen, val_gen)

# Save the trained model
cnn_model.save('abnormality_classifier_vgg16.h5')

# Verify the dataset preparation
print(f"Classes found: {train_gen.class_indices}")
print(f"Number of training samples: {train_gen.samples}")
print(f"Number of validation samples: {val_gen.samples}")

Found 228 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
Epoch 1/20


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step - accuracy: 0.5240 - loss: 0.7980 - val_accuracy: 0.1429 - val_loss: 0.9788
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.4831 - loss: 0.7567 - val_accuracy: 0.4821 - val_loss: 0.6176
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.6892 - loss: 0.5706 - val_accuracy: 0.5000 - val_loss: 0.6542
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step - accuracy: 0.6394 - loss: 0.6539 - val_accuracy: 0.5536 - val_loss: 0.6552
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.7012 - loss: 0.5873 - val_accuracy: 0.5536 - val_loss: 0.6164
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.7821 - loss: 0.4184 - val_accuracy: 0.5536 - val_loss: 0.6762
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.7581 - loss: 0.4225 - val_accuracy: 0.4821 - val_loss: 0.9584
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.7641 - loss: 0.4192 - val_accuracy: 0.5536 - val_loss: 0.8043
Epoch 9/20
8/8 ━━━━

Classes found: {'Fire': 0, 'Smoke': 1}
Number of training samples: 228
Number of validation samples: 56


In [20]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (128, 128))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function to include classification and saving frames
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(cnn_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function
def motion_detection(video_path, output_dir, cnn_model, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_end_time = frame_number / original_fps
                abnormality_duration_seconds = abnormality_end_time - (abnormality_start_frame / original_fps)

                print(f"Abnormality {abnormality_group_id} start time: {int((abnormality_start_frame / original_fps) // 60)} minutes {(abnormality_start_frame / original_fps) % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

                abnormality_detected = False

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Load the trained model
cnn_model = load_model('abnormality_classifier_vgg16.h5')

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\2-DMG_RSA_HTE101_D_HS_FIRE_31-33min\output31.mp4"
output_dir = r"C:\Users\pkathi\Desktop\my work\computervision-video\augmentvgg_1"
abnormality_times = motion_detection(video_path, output_dir, cnn_model)
print("Abnormality frames saved in the output directory.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step
Smoke detected at 0m_51.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Smoke detected at 0m_51.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
Smoke detected at 0m_51.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
Smoke detected at 0m_51.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
Smoke detected at 0m_51.80s
Abnormality 1 start time: 0 minutes 51.00 seconds
Abnormality 1 duration: 0 minutes 1.00 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
Smoke detected at 0m_52.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
Smoke detected at 0m_53.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
Smoke detected at 0m_53.20s
Abnormality 1 start time: 0 minutes 52.80 seconds
Abnormality 1 duration: 0 minutes 0.60 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
Smoke detected at 0m_55.20s
Abnormality 1 start time: 0 minutes 55.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Smoke detected at 0m_56.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Load and preprocess dataset with data augmentation
def prepare_dataset(data_dir, target_size=(128, 128), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = validation_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Define the model using transfer learning (VGG16)
def create_transfer_learning_model(input_shape, num_classes):
    base_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model with class weighting
def train_model(model, train_generator, validation_generator, epochs=20):
    # Extract class indices from the training generator
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    classes = np.array([class_indices[i] for i in train_generator.classes])
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
    class_weights_dict = dict(enumerate(class_weights))

    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        class_weight=class_weights_dict
    )

# Example usage
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset2-minimal'
input_shape = (128, 128, 3)
num_classes = 2

train_gen, val_gen = prepare_dataset(data_dir, target_size=(128, 128))
cnn_model = create_transfer_learning_model(input_shape, num_classes)
train_model(cnn_model, train_gen, val_gen)

# Save the trained model
cnn_model.save('abnormality_classifier_vgg16.h5')

# Verify the dataset preparation
print(f"Classes found: {train_gen.class_indices}")
print(f"Number of training samples: {train_gen.samples}")
print(f"Number of validation samples: {val_gen.samples}")

Found 56 images belonging to 2 classes.
Found 13 images belonging to 2 classes.
Epoch 1/20


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5893 - loss: 0.6971 - val_accuracy: 0.6154 - val_loss: 0.8226
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.4077 - loss: 0.9131 - val_accuracy: 0.6154 - val_loss: 0.7983
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.6131 - loss: 0.6896 - val_accuracy: 0.6154 - val_loss: 0.7609
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.6711 - loss: 0.6860 - val_accuracy: 0.6154 - val_loss: 0.7318
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.5893 - loss: 0.6986 - val_accuracy: 0.6154 - val_loss: 0.7414
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.7277 - loss: 0.5306 - val_accuracy: 0.6154 - val_loss: 0.7049
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.6577 - loss: 0.7210 - val_accuracy: 0.6154 - val_loss: 0.6566
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.6865 - loss: 0.6056 - val_accuracy: 0.6154 - val_loss: 0.6291
Epoch 9/20
2/2 ━━━━━━━━━━━━

Classes found: {'Fire': 0, 'Smoke': 1}
Number of training samples: 56
Number of validation samples: 13


In [4]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (128, 128))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function to include classification and saving frames
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(cnn_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function
def motion_detection(video_path, output_dir, cnn_model, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_end_time = frame_number / original_fps
                abnormality_duration_seconds = abnormality_end_time - (abnormality_start_frame / original_fps)

                print(f"Abnormality {abnormality_group_id} start time: {int((abnormality_start_frame / original_fps) // 60)} minutes {(abnormality_start_frame / original_fps) % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

                abnormality_detected = False

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Load the trained model
cnn_model = load_model('abnormality_classifier_vgg16.h5')

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = r"C:\Users\pkathi\Desktop\my work\computervision-video\augmentvgg_3minima"
abnormality_times = motion_detection(video_path, output_dir, cnn_model)
print("Abnormality frames saved in the output directory.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
Smoke detected at 0m_57.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Smoke detected at 0m_57.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Smoke detected at 0m_58.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Smoke detected at 0m_58.20s
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Smoke detected at 0m_58.60s
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Fire detected at 1m_3.00s
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Fire detected at 1m_3.40s
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Smoke detected at 1m_17.00s
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minute

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Load and preprocess dataset with data augmentation
def prepare_dataset(data_dir, target_size=(128, 128), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = validation_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Define the model using transfer learning (VGG16)
def create_transfer_learning_model(input_shape, num_classes):
    base_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model with class weighting
def train_model(model, train_generator, validation_generator, epochs=20):
    # Extract class indices from the training generator
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    classes = np.array([class_indices[i] for i in train_generator.classes])
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
    class_weights_dict = dict(enumerate(class_weights))

    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        class_weight=class_weights_dict
    )

# Example usage
data_dir = r"C:\Users\pkathi\Desktop\my work\object detection\tradianal approach-cv\Dataset2-minimal"
input_shape = (128, 128, 3)
num_classes = 2

train_gen, val_gen = prepare_dataset(data_dir, target_size=(128, 128))
cnn_model = create_transfer_learning_model(input_shape, num_classes)
train_model(cnn_model, train_gen, val_gen)

# Save the trained model
cnn_model.save('abnormality_classifier_vgg16.h5')

# Verify the dataset preparation
print(f"Classes found: {train_gen.class_indices}")
print(f"Number of training samples: {train_gen.samples}")
print(f"Number of validation samples: {val_gen.samples}")

Found 46 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


Epoch 1/20


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.4896 - loss: 0.9173 - val_accuracy: 0.8000 - val_loss: 0.6807
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.6491 - loss: 0.7740 - val_accuracy: 0.8000 - val_loss: 0.6683
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.5673 - loss: 0.8687 - val_accuracy: 0.2000 - val_loss: 0.9383
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.4037 - loss: 1.1092 - val_accuracy: 0.2000 - val_loss: 1.3336
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.4420 - loss: 0.8384 - val_accuracy: 0.2000 - val_loss: 1.0675
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.4576 - loss: 0.6961 - val_accuracy: 0.4000 - val_loss: 0.6322
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.7164 - loss: 0.5985 - val_accuracy: 1.0000 - val_loss: 0.3653
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.8320 - loss: 0.3998 - val_accuracy: 1.0000 - val_loss: 0.2819
Epoch 9/20
2/2 ━━━━━━━━━━━━

Classes found: {'Fire': 0, 'Smoke': 1}
Number of training samples: 46
Number of validation samples: 10


In [6]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (128, 128))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function to include classification and saving frames
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(cnn_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function
def motion_detection(video_path, output_dir, cnn_model, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_end_time = frame_number / original_fps
                abnormality_duration_seconds = abnormality_end_time - (abnormality_start_frame / original_fps)

                print(f"Abnormality {abnormality_group_id} start time: {int((abnormality_start_frame / original_fps) // 60)} minutes {(abnormality_start_frame / original_fps) % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

                abnormality_detected = False

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Load the trained model
cnn_model = load_model('abnormality_classifier_vgg16.h5')

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = r"C:\Users\pkathi\Desktop\my work\computervision-video\augmentvgg_4minima"
abnormality_times = motion_detection(video_path, output_dir, cnn_model)
print("Abnormality frames saved in the output directory.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
Fire detected at 0m_57.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Fire detected at 0m_57.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Fire detected at 0m_58.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Fire detected at 0m_58.20s
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Fire detected at 0m_58.60s
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Fire detected at 1m_3.00s
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Fire detected at 1m_3.40s
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Smoke detected at 1m_17.00s
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.2

In [4]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from multiprocessing.pool import ThreadPool

# Use the model to classify abnormalities
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (128, 128))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function to include classification and saving frames
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(cnn_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function
def motion_detection(video_path, output_dir, cnn_model, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, cnn_model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_end_time = frame_number / original_fps
                abnormality_duration_seconds = abnormality_end_time - (abnormality_start_frame / original_fps)

                print(f"Abnormality {abnormality_group_id} start time: {int((abnormality_start_frame / original_fps) // 60)} minutes {(abnormality_start_frame / original_fps) % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

                abnormality_detected = False

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Load the trained model
cnn_model = load_model('abnormality_classifier_vgg16.h5')

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\2-DMG_RSA_HTE101_D_HS_FIRE_31-33min\output31.mp4"
output_dir = r"C:\Users\pkathi\Desktop\my work\computervision-video\augmentvgg_5minima"
abnormality_times = motion_detection(video_path, output_dir, cnn_model)
print("Abnormality frames saved in the output directory.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
Smoke detected at 0m_51.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Smoke detected at 0m_51.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Smoke detected at 0m_51.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Smoke detected at 0m_51.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
Smoke detected at 0m_51.80s
Abnormality 1 start time: 0 minutes 51.00 seconds
Abnormality 1 duration: 0 minutes 1.00 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Smoke detected at 0m_52.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Smoke detected at 0m_53.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Smoke detected at 0m_53.20s
Abnormality 1 start time: 0 minutes 52.80 seconds
Abnormality 1 duration: 0 minutes 0.60 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Smoke detected at 0m_55.20s
Abnormality 1 start time: 0 minutes 55.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Smoke detected at 0m_56.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


In [7]:
#Resnet model

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Load and preprocess dataset with data augmentation
def prepare_dataset(data_dir, target_size=(128, 128), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = validation_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Define the model using transfer learning (ResNet50)
def create_transfer_learning_model(input_shape, num_classes):
    base_model = ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model with class weighting
def train_model(model, train_generator, validation_generator, epochs=20):
    # Extract class indices from the training generator
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    classes = np.array([class_indices[i] for i in train_generator.classes])
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
    class_weights_dict = dict(enumerate(class_weights))

    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        class_weight=class_weights_dict
    )

# Example usage
data_dir = r'C:\Users\pkathi\Desktop\object detection\tradianal approach-cv\Dataset2-minimal'
input_shape = (128, 128, 3)
num_classes = 2

train_gen, val_gen = prepare_dataset(data_dir, target_size=(128, 128))
resnet_model = create_transfer_learning_model(input_shape, num_classes)
train_model(resnet_model, train_gen, val_gen)

# Save the trained model
resnet_model.save('abnormality_classifier_resnet50.h5')

# Verify the dataset preparation
print(f"Classes found: {train_gen.class_indices}")
print(f"Number of training samples: {train_gen.samples}")
print(f"Number of validation samples: {val_gen.samples}")

Found 46 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/20


C:\Users\pkathi\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.3859 - loss: 0.9660 - val_accuracy: 0.8000 - val_loss: 0.5785
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 795ms/step - accuracy: 0.6431 - loss: 0.8809 - val_accuracy: 0.6000 - val_loss: 0.6950
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 591ms/step - accuracy: 0.3361 - loss: 0.8700 - val_accuracy: 0.1000 - val_loss: 0.7538
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 752ms/step - accuracy: 0.4461 - loss: 0.8372 - val_accuracy: 0.2000 - val_loss: 0.7642
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.4669 - loss: 0.8076 - val_accuracy: 0.1000 - val_loss: 0.7184
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5249 - loss: 0.8438 - val_accuracy: 0.6000 - val_loss: 0.6928
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 939ms/step - accuracy: 0.5892 - loss: 0.6610 - val_accuracy: 0.8000 - val_loss: 0.6492
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.3509 - loss: 0.8278 - val_accuracy: 0.8000 - val_loss: 0.6581
Epoch 9/20
2/2

Classes found: {'Fire': 0, 'Smoke': 1}
Number of training samples: 46
Number of validation samples: 10


In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
from multiprocessing.pool import ThreadPool

# Load the trained model
resnet_model = load_model('abnormality_classifier_resnet50.h5')

# Classify the abnormality in a frame
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (128, 128))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function to include classification and saving frames
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(resnet_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function remains the same
def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps
            if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                abnormality_group_id += 1

            abnormality_start_frame = frame_number
            abnormality_detected = True
            abnormality_times.append(frame_number)
            last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_end_time = frame_number / original_fps
                abnormality_duration_seconds = abnormality_end_time - (abnormality_start_frame / original_fps)

                print(f"Abnormality {abnormality_group_id} start time: {int((abnormality_start_frame / original_fps) // 60)} minutes {(abnormality_start_frame / original_fps) % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

                abnormality_detected = False

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = r"C:\Users\pkathi\Desktop\my work\computervision-video\augmentresnet_1minima"
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Fire detected at 0m_57.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Fire detected at 0m_57.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Fire detected at 0m_58.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Fire detected at 0m_58.20s
Abnormality 1 start time: 0 minutes 58.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Fire detected at 0m_58.60s
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Fire detected at 1m_3.00s
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Fire detected at 1m_3.40s
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Fire detected at 1m_17.00s
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1

In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
from multiprocessing.pool import ThreadPool

# Load the trained model
resnet_model = load_model('abnormality_classifier_resnet50.h5')

# Classify the abnormality in a frame
def classify_abnormality(model, frame):
    frame_resized = cv2.resize(frame, (128, 128))  # Resize to match model input shape
    frame_normalized = frame_resized / 255.0  # Normalize the image
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Expand dimensions to match batch size
    prediction = model.predict(frame_expanded)
    classes = ['Fire', 'Smoke']
    return classes[np.argmax(prediction)]

# Update the process_frame function to include classification and saving frames
def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        # Classify the abnormality
        classification = classify_abnormality(resnet_model, frame_resized)
        
        abnormality_time = frame_number / original_fps
        timestamp = f"{int(abnormality_time // 60)}m_{abnormality_time % 60:.2f}s"
        output_frame_path = os.path.join(output_dir, classification, f"{classification}_{timestamp}_frame_{frame_number}.jpg")

        os.makedirs(os.path.dirname(output_frame_path), exist_ok=True)
        cv2.imwrite(output_frame_path, frame_resized)

        print(f"{classification} detected at {timestamp}")
    
    return detected, frame_gray

# Motion detection function remains the same
def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps
            if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                abnormality_group_id += 1

            abnormality_start_frame = frame_number
            abnormality_detected = True
            abnormality_times.append(frame_number)
            last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_end_time = frame_number / original_fps
                abnormality_duration_seconds = abnormality_end_time - (abnormality_start_frame / original_fps)

                print(f"Abnormality {abnormality_group_id} start time: {int((abnormality_start_frame / original_fps) // 60)} minutes {(abnormality_start_frame / original_fps) % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

                abnormality_detected = False

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\2-DMG_RSA_HTE101_D_HS_FIRE_31-33min\output31.mp4"
output_dir = r"C:\Users\pkathi\Desktop\my work\computervision-video\augmentresnet_2minima"
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Fire detected at 0m_51.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Fire detected at 0m_51.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Fire detected at 0m_51.40s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Fire detected at 0m_51.60s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Fire detected at 0m_51.80s
Abnormality 1 start time: 0 minutes 51.80 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Fire detected at 0m_52.80s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Fire detected at 0m_53.00s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Fire detected at 0m_53.20s
Abnormality 1 start time: 0 minutes 53.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Fire detected at 0m_55.20s
Abnormality 1 start time: 0 minutes 55.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Fire detected at 0m_56.20s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Fire detected at 0m_56.